In [4]:
import sys
sys.path.append('..')

import json
import ollama.client as client

model = 'zephyr'

# response, _ = client.generate(model_name=model,prompt='prompt' )
client.show('zephyr:latest')

{'modelfile': '# Modelfile generated by "ollama show"\n# To build a new Modelfile based on this one, replace the FROM line with:\n# FROM zephyr:latest\n\nFROM /Users/rahulnayak/.ollama/models/blobs/sha256:0e655574a746a702b8faa09ea5bd779801371cec9e34f69760d0ebb1aec1a6ce\nTEMPLATE """{{- if .System }}\n<|system|>\n{{ .System }}\n</s>\n{{- end }}\n<|user|>\n{{ .Prompt }}\n</s>\n<|assistant|>\n"""\nPARAMETER stop "<|system|>"\nPARAMETER stop "<|user|>"\nPARAMETER stop "<|assistant|>"\nPARAMETER stop "</s>"',
 'parameters': 'stop                           <|system|>\nstop                           <|user|>\nstop                           <|assistant|>\nstop                           </s>',
 'template': '{{- if .System }}\n<|system|>\n{{ .System }}\n</s>\n{{- end }}\n<|user|>\n{{ .Prompt }}\n</s>\n<|assistant|>\n'}

In [20]:

def extractConcepts(prompt: str, metadata = {}, model=model):
    SYS_PROMPT = (
        "Your task is extract the key concepts (and non personal entities) mentioned in the given text. "
        "Extract only the most important and atomistic concepts, if  needed break the concepts down to the simpler concepts."
        "Categorize the concepts in one of the following categories: "
        "[event, concept, place, object, document, organisation, condition, misc]\n"
        "Format your output as a list of json with the following format:\n"
        "[\n"
        "   {\n"
        '       "entity": The Concept,\n'
        '       "importance": The contextual importance of the concept on a scale of 1 to 5 (5 being the highest),\n'
        '       "category": The Type of Concept,\n'
        "   }, \n"
        "{ }, \n"
        "]\n"
    )
    response, _ = client.generate(
        model_name=model, system=SYS_PROMPT, prompt=prompt
    )
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    return result

def graphPrompt(input: str, metadata={}, model=model):
    model_info = client.show(model_name=model)
    print(json.loads(model_info), "\n\n")
    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given text"
        "Your task is to extract the ontology of terms mentioned in the given text. "
        "Thought 1: While travelling through every sentence, Think about the key terms mentioned in it.\n"
                    "\tTerms may include object, entity, location, organization, person, \n"
                    "\tcondition, acronym, documents, service, concept, etc.\n"
                    "\tTerms should be as atomistic as possible\n\n"
        "Thought 3: Think about how these terms can have one on one relation with other terms.\n"
                    "\tTerms that are mentioned in same sentence or the same paragraph are typically related to each other.\n"
                    "\tTerms can be related to many other terms\n\n"
        "Thought 4: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        "       \"node_1\": \"A concept from extracted ontology\",\n"
        "       \"node_2\": \"A related concept from extracted ontology\",\n"
        "       \"edge\": \"relationship between the two concepts, node_1 and node_2 in one or two sentences\"\n"
        "   }, {...}\n"
        "]"
    )
    response, _ = client.generate(
        model_name=model, 
        system=SYS_PROMPT, 
        # prompt=SYS_PROMPT + "\n\n" + input
        prompt=input
        )
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    return result

In [21]:
r = graphPrompt(
    """
text delimites bweteen ``` \n
text: ```
Mental Health as a Pathway to Health Equityargued that investing in mental health is important from the point of view 
of improving population health and reducing social inequality, promoting 
human rights, and improving economic efficiency.8 The economic implications 
of mental and substance disorders can be enormous: for example, the 
global economic loss due to common mental disorders is an estimated US$ 
1 trillion per year.9 Given the increasing prevalence of mental illness, the 
costs are likely to only further mount. Investment in treatment of mental 
illness can result in large economic productivity gains.10
The Burden of Mental Health, Substance Use Disorders and 
Multimorbidity
Mental and substance-abuse disorders are leading contributors to premature 
morbidity and mortality globally. In India, for example, the National Mental 
Health Survey (NMHS), 2015-16, has reported a 13.6-percent lifetime 
prevalence of any mental/substance-abuse disorder morbidity.11 The overall 
number of Disability-Adjusted-Life-Years (DALYs) for mental disorders has 
increased by more than 50 percent between 1990 and 2019.12 This growth 
pattern is expected to continue, necessitating a proportionate response from 
the health systems. Most of the mental and substance abuse disorders 
are chronic and disabling conditions. The treatment gap for mental and 
substance abuse disorders (defined as proportions of those needing care
```
"""
)

{'modelfile': '# Modelfile generated by "ollama show"\n# To build a new Modelfile based on this one, replace the FROM line with:\n# FROM zephyr:latest\n\nFROM /Users/rahulnayak/.ollama/models/blobs/sha256:0e655574a746a702b8faa09ea5bd779801371cec9e34f69760d0ebb1aec1a6ce\nTEMPLATE """{{- if .System }}\n<|system|>\n{{ .System }}\n</s>\n{{- end }}\n<|user|>\n{{ .Prompt }}\n</s>\n<|assistant|>\n"""\nPARAMETER stop "<|system|>"\nPARAMETER stop "<|user|>"\nPARAMETER stop "<|assistant|>"\nPARAMETER stop "</s>"', 'parameters': 'stop                           <|system|>\nstop                           <|user|>\nstop                           <|assistant|>\nstop                           </s>', 'template': '{{- if .System }}\n<|system|>\n{{ .System }}\n</s>\n{{- end }}\n<|user|>\n{{ .Prompt }}\n</s>\n<|assistant|>\n'}
[
   {
       "node_1": "Mental Health",
       "node_2": "Health Equity",
       "edge": "Investing in mental health is important from the point of view improving population health

KeyboardInterrupt: 